The code below is adapted from the ANN In-Class Activity.

The model takes as input ventricle and hippocampal volume and predicts whether or not an individual has Alzheimer's, is mildly cognitively impaired, or cognitively normal based on scores on the ADAS.

The reported model accuracy is 0.00117, which is very inaccurate. Sources of error and model inaccuracy are discussed in the Error section.

In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.2f}".format

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ADNIMERGEcolab.csv to ADNIMERGEcolab (1).csv
User uploaded file "ADNIMERGEcolab.csv" with length 6836925 bytes


In [ ]:
dataset = pd.read_csv("ADNIMERGEcolab.csv", header='infer')
dataset.head()

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (20,90,91) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,RID,VISCODE,COLPROT,ORIGPROT,EXAMDATE,DX.bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,...,ABETA.bl,PTAU.bl,TAU.bl,FDG.bl,PIB.bl,AV45.bl,Years.bl,Month.bl,Month,M
0,2,bl,ADNI1,ADNI1,9/8/05,CN,74.30,Male,16,Not Hisp/Latino,...,NaN,NaN,NaN,1.37,NaN,NaN,0.00,0.00,0,0
1,2,m06,ADNI1,ADNI1,3/6/06,CN,74.30,Male,16,Not Hisp/Latino,...,NaN,NaN,NaN,1.37,NaN,NaN,0.49,5.87,6,6
2,2,m36,ADNI1,ADNI1,8/27/08,CN,74.30,Male,16,Not Hisp/Latino,...,NaN,NaN,NaN,1.37,NaN,NaN,2.97,35.54,36,36
3,2,m60,ADNIGO,ADNI1,9/22/10,CN,74.30,Male,16,Not Hisp/Latino,...,NaN,NaN,NaN,1.37,NaN,NaN,5.04,60.33,60,60
4,2,m66,ADNIGO,ADNI1,3/4/11,CN,74.30,Male,16,Not Hisp/Latino,...,NaN,NaN,NaN,1.37,NaN,NaN,5.48,65.67,66,66


In [ ]:
dataset.columns
dataset[['Ventricles.bl', 'Hippocampus.bl']].describe

<bound method NDFrame.describe of        Ventricles.bl  Hippocampus.bl
0          118233.00         8336.00
1          118233.00         8336.00
2          118233.00         8336.00
3          118233.00         8336.00
4          118233.00         8336.00
...              ...             ...
12744            NaN         5928.00
12745            NaN         5928.00
12746            NaN         5928.00
12747       18633.00        10602.00
12748       18633.00        10602.00

[12749 rows x 2 columns]>

In [ ]:
dataset['ADAS11.bl'].describe()

count   12732.00
mean        9.81
std         5.91
min         0.00
25%         5.67
50%         8.67
75%        13.00
max        42.67
Name: ADAS11.bl, dtype: float64

In [ ]:
dataset['AD'] = dataset.apply(lambda x: (x['ADAS11.bl']>=15), axis=1)
dataset['MCI'] = dataset.apply(lambda x: (x['ADAS11.bl']>=10 and x['ADAS11.bl']<15), axis=1)
dataset['CN'] = dataset.apply(lambda x: (x['ADAS11.bl']>=0 and x['ADAS11.bl']<10), axis=1)

In [ ]:
dataset[['AD', 'MCI', 'CN']]

,AD,MCI,CN
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False
...,...,...,...
12744,False,True,False
12745,False,True,False
12746,False,True,False
12747,False,False,True


In [ ]:
X = dataset[['Ventricles.bl', 'Hippocampus.bl']]
y = dataset[['AD', 'MCI', 'CN']]
print(X)
print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0, stratify=y)

       Ventricles.bl  Hippocampus.bl
0          118233.00         8336.00
1          118233.00         8336.00
2          118233.00         8336.00
3          118233.00         8336.00
4          118233.00         8336.00
...              ...             ...
12744            NaN         5928.00
12745            NaN         5928.00
12746            NaN         5928.00
12747       18633.00        10602.00
12748       18633.00        10602.00

[12749 rows x 2 columns]
          AD    MCI     CN
0      False   True  False
1      False   True  False
2      False   True  False
3      False   True  False
4      False   True  False
...      ...    ...    ...
12744  False   True  False
12745  False   True  False
12746  False   True  False
12747  False  False   True
12748  False  False   True

[12749 rows x 3 columns]


In [ ]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=18, activation='relu'))
ann.add(tf.keras.layers.Dense(units=18,activation='relu'))
ann.add(tf.keras.layers.Dense(units=3, activation='sigmoid'))

In [ ]:
ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
319/319 [==============================] - 3s 3ms/step - loss: nan - accuracy: 0.1742
Epoch 2/100
319/319 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.1734
Epoch 3/100
319/319 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.1734
Epoch 4/100
319/319 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.1734
Epoch 5/100
319/319 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.1734
Epoch 6/100
319/319 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.1734
Epoch 7/100
319/319 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.1734
Epoch 8/100
319/319 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.1734
Epoch 9/100
319/319 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.1734
Epoch 10/100
319/319 [==============================] - 1s 4ms/step - loss: nan - accuracy: 0.1734
Epoch 11/100
319/31

In [ ]:
y_pred = ann.predict(X_test)
y_pred = np.array(y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),3), np.array(y_test).reshape(len(y_test),3)),))

80/80 [==============================] - 0s 1ms/step
[[False False False]
 [False False False]
 [False False False]
 ...
 [False False  True]
 [False  True False]
 [False False  True]]


In [ ]:
acc = accuracy_score(y_test, y_pred)
print(acc)

0.001176470588235294
